In [1]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from google.cloud import vision
from google.cloud import storage
from PIL import Image, ImageSequence
from gensim.models import Doc2Vec
from IPython.core.debugger import set_trace
import IPython
import collections
from googletrans import Translator
from google.cloud import translate_v2 as translate

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value
from django.apps import apps
from django_currentuser.middleware import get_current_user
from datetime import datetime, timedelta
import time
import pandas as pd

In [2]:
# trans = m.Transaction.objects.select_related('sender','receiver'); trans
trans = m.Transaction.objects.select_related(
    'sender','sender__boo','sender__boo__user','sender__raffle','sender__raffle__item','sender__support','sender__support__brand',
    'receiver','receiver__boo','receiver__boo__user','receiver__raffle','receiver__raffle__item','receiver__support','receiver__support__brand'
); trans

<QuerySet [<Transaction: quantlab | quantlab@kakao.com > KINTO TUMBLER 350ML - 39000 | 2021-08-07 20:38:17>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > STARBUCKS LAMP BK - 0 | 2021-08-07 20:37:53>, <Transaction: quantlab | quantlab@kakao.com > TAW&TOE ZEROVITY™ Flip Flop, SIZE M - 38000 | 2021-08-07 23:34:16>, <Transaction: quantlab | quantlab@kakao.com > KINTO TUMBLER 350ML - 39000 | 2021-08-07 20:38:17>, <Transaction: quantlab | quantlab@kakao.com > JOHNBER | 존버 스포츠 | 2021-08-10 18:02:20>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>,

In [63]:
# trans_dict = [{ 
#     'sender': str(_t.sender),
#     'receiver': str(_t.receiver),
#     'when': str(_t.when.date()),
#     'type': _t.get_type_display(),
#     'amount': _t.amount
# } for _t in trans[:10]]; trans_dict

In [3]:
trans_list = []
for _t in trans:
    tr = {}
    tr_type = _t.get_type_display()
    tr['user'] = str(_t.sender) if tr_type in ['래플','후원'] else str(_t.receiver)
    tr['when'] = str(_t.when.date())
    tr['type'] = tr_type
    tr['amount'] = -_t.amount if tr_type in ['래플','후원'] else _t.amount
    trans_list.append(tr)

In [82]:
# pd.DataFrame(trans_list).groupby(['user','type','when']).sum().unstack().fillna(0).to_excel('transactions_amount.xlsx')

In [85]:
# pd.DataFrame(trans_list).groupby(['user','type','when']).count().unstack().fillna(0).to_excel('transactions_count.xlsx')

In [19]:
counts = pd.DataFrame(trans_list).groupby(['user','type','when']).count().unstack().fillna(0)
counts = counts.groupby('type').sum()[counts.columns[-15:-1]].drop(['첫방문환영']).astype('int64')
counts.style.set_table_attributes('style="font-size: 10px"')

In [17]:
amounts = pd.DataFrame(trans_list).groupby(['user','type','when']).sum().unstack().fillna(0)
amounts = amounts.groupby('type').sum()[counts.columns[-15:-1]].drop(['첫방문환영']).astype('int64')
amounts.style.set_table_attributes('style="font-size: 10px"')

In [7]:
qrs = m.User.objects.filter(date_joined__gte='2021-09-13').values('date_joined__date').annotate(count=Count('date_joined__date'))

In [20]:
pd.DataFrame(qrs).set_index('date_joined__date').sort_index(ascending=True).iloc[:-1]

,count
date_joined__date,
2021-09-13,9
2021-09-14,3
2021-09-15,7
2021-09-16,14
2021-09-17,13
2021-09-18,10
2021-09-19,7
2021-09-20,6
2021-09-21,12


In [27]:
_boos = m.Boo.objects.filter(answers__has_key='12')

In [32]:
_boos[10].answers

{'7': {'47': [0],
  '48': [1],
  '49': [1],
  '50': [0],
  '51': [0],
  '172': [0],
  '173': [1],
  '174': [0],
  '176': [1],
  '177': [0],
  '178': [0],
  '180': [1],
  '181': [1],
  '182': [1],
  '183': [0],
  'finished': True},
 '12': {'80': [0],
  '81': [1],
  '82': [0],
  '83': [1],
  '84': [1],
  '85': ['티'],
  '86': ['ㅇ'],
  '87': [1],
  '88': [1],
  '89': [1],
  'finished': True},
 '19': {'238': [1], '239': [0], '240': [0], 'finished': True}}

In [ ]:
배드보이 4
터치그라운드 5
존버 6
포스토 7
브라스보트 8

비오는날 취향분석 9
슬기로운 캠필생활 10
나에게 딱 맞는 주말 베스트 코디 12
내 휴가 스타일 14
핫서머에 입기 좋은 핫팬츠 취향조사 15
구두취향 본격탐구 17
내 백 취향 18
요즘 막 당신의 관심사 19
비오는날 샌들 취향조사 20
어느 브랜드 좋아해 22
올가을 트랜드 조사 신발편 23
가을 유니폼 후디 맨투맨 고르기 24
부츠 이렇게 신어야지 25
이번 fw 뭐 신지 26


In [33]:
m.Boo.objects.filter(answers__has_key='4').count()

67

In [47]:
pd.DataFrame({
    '배드보이': [m.Boo.objects.filter(answers__has_key='4').count()],
    '터치그라운드': [m.Boo.objects.filter(answers__has_key='5').count()],
    '존버': [m.Boo.objects.filter(answers__has_key='6').count()],
    '포스토': [m.Boo.objects.filter(answers__has_key='7').count()],
    '브라스보트': [m.Boo.objects.filter(answers__has_key='8').count()],
    
    '비오는날 취향분석': [m.Boo.objects.filter(answers__has_key='9').count()],
    '슬기로운 캠핑생활': [m.Boo.objects.filter(answers__has_key='10').count()],
    '주말 베스트 코디': [m.Boo.objects.filter(answers__has_key='12').count()],
    '내 휴가 스타일': [m.Boo.objects.filter(answers__has_key='14').count()],
    '핫팬츠 취향조사': [m.Boo.objects.filter(answers__has_key='15').count()],
    '구두취향 본격탐구': [m.Boo.objects.filter(answers__has_key='17').count()],
    '내 백 취향': [m.Boo.objects.filter(answers__has_key='18').count()],
    '요즘 막 당신의 관심사': [m.Boo.objects.filter(answers__has_key='19').count()],
    '샌들 취향조사': [m.Boo.objects.filter(answers__has_key='20').count()],
    '어느 브랜드 좋아해': [m.Boo.objects.filter(answers__has_key='22').count()],
    '올가을 트랜드 조사 신발편': [m.Boo.objects.filter(answers__has_key='23').count()],
    '가을 유니폼 후디 맨투맨 고르기': [m.Boo.objects.filter(answers__has_key='24').count()],
    '부츠 이렇게 신어야지': [m.Boo.objects.filter(answers__has_key='25').count()]
}).T

,0
배드보이,67
터치그라운드,53
존버,47
포스토,67
브라스보트,53
비오는날 취향분석,53
슬기로운 캠핑생활,56
주말 베스트 코디,60
내 휴가 스타일,33
핫팬츠 취향조사,16


In [39]:
pd.DataFrame({'a':[1], 'b':[2]})

,a,b
0,1,2
